## Before you start:

First, make sure you are using the conda environment `pope` that is included in the `popping-off` repo.

Second, the cell block below loads the data, using the data path that is given in `popping-off/data_paths.json`. 
You need to customise the following:

- `base_path`: path of the `.pkl` data set. (e.g., `sessions_lite_flu_2021-10-22.pkl`)
- `vape_path`: path of the `Vape` repo.

In [1]:
## Following lines such that modules are automatically updated:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import popoff  # custom package with all code routines
import pop_off_plotting as pop  # module with plotting functions
import pop_off_functions as pof  # module with analysis functions
from Session import SessionLite  # Class object that holds data of 1 recording session
from linear_model import PoolAcrossSessions, LinearModel, MultiSessionModel  # Classes that hold multiple Sessions

/home/tplas/repos/popping-off/popoff/popoff/loadpaths.py
/home/tplas/repos/Vape


## Load data:

We load all recording sessions into a joint object `PoolAcrossSessions()`. 
This object contains each sessions as individual `SessionLite()` classes, which we extract and put in the dictionary `sessions`. 

In [4]:
## Load data in form of Jimmy's PAS object
pas = PoolAcrossSessions(save_PCA=False, subsample_sessions=False,
                         remove_targets=False, remove_toosoon=True)

## Create sessions object from PAS:
sessions = {}
int_keys_pas_sessions = pas.sessions.keys()
for i_s, ses in enumerate(pas.sessions.values()):  # load into sessions dict (in case pas skips an int as key)
    ses.signature = f'{ses.mouse}_R{ses.run_number}'
    sessions[i_s] = ses
print(sessions)
assert len(sessions) == 11
pof.label_urh_arm(sessions=sessions)  # label arm and urh trials

print('------------------------------------')
print(f'{len(sessions)} sessions are loaded')
# tp_dict = pof.create_tp_dict(sessions=sessions)


long post time
long post time
long post time
long post time
long post time
Mouse RL070, run 29  registered no-lick hit. changed to too soon
long post time
long post time
Mouse RL117, run 29  registered no-lick hit. changed to too soon
Mouse RL117, run 29  registered no-lick hit. changed to too soon
long post time
long post time
long post time
Mouse RL116, run 32  registered no-lick hit. changed to too soon
Mouse RL116, run 32  registered no-lick hit. changed to too soon
Mouse RL116, run 32  registered no-lick hit. changed to too soon
long post time
ALERT SESSIONS NOT SUBSAMPLED
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time
{0: instance Mouse J064, run 10 of Session class, 1: instance Mouse J064, run 11 of Session class, 2: instance Mouse J064, run 14 of Session class, 3: instance Mouse RL070, run 28 of Session class, 4: instance Mouse RL070, run 29 of Session class, 5:

## Load a single recording session

Sessions are indexed by ints.

In [4]:
example_session = sessions[0]  # get the first session

## Identify the session:
print(f'Full name: {example_session.name}. Short name (that we often use as identifier): {example_session.signature}')

## Some important data objects:
## Number of cells, and region identity:
print(f'Session {example_session.signature} has {example_session.n_cells} cells.')
print(f'Of these, {example_session.s1_bool.sum()} are in S1 and {example_session.s2_bool.sum()} in S2')
assert example_session.s1_bool.sum() + example_session.s2_bool.sum() == example_session.n_cells 

## Number of trial, and stimulation:
print(f'There are {example_session.n_trials} trials in total.')
assert example_session.n_trials == len(example_session.trial_subsets)
for n_stim in np.unique(example_session.trial_subsets):
    print(f'There are {np.sum(example_session.trial_subsets == n_stim)} trials with {n_stim} cells stimulated')

## Number of trials, and outcomes:
assert example_session.n_trials == len(example_session.outcome)
for behav_outcome in np.unique(example_session.outcome):
    print(f'There are {np.sum(example_session.outcome == behav_outcome)} trials with outcome {behav_outcome}')


Full name: Mouse J064, run 10. Short name (that we often use as identifier): J064_R10
Session J064_R10 has 471 cells.
Of these, 319 are in S1 and 152 in S2
There are 162 trials in total.
There are 61 trials with 0 cells stimulated
There are 9 trials with 5 cells stimulated
There are 8 trials with 10 cells stimulated
There are 7 trials with 20 cells stimulated
There are 13 trials with 30 cells stimulated
There are 8 trials with 40 cells stimulated
There are 9 trials with 50 cells stimulated
There are 47 trials with 150 cells stimulated
There are 2 trials with outcome arm
There are 31 trials with outcome cr
There are 28 trials with outcome fp
There are 66 trials with outcome hit
There are 16 trials with outcome miss
There are 13 trials with outcome too_
There are 6 trials with outcome urh


In [5]:
## Data:

# example_session.filter_ps_time  # time array of a trial, in seconds 
# example_session.filter_ps_array  # frame array of  trial, in frames
assert example_session.filter_ps_time.shape == example_session.filter_ps_array.shape

## Neural data:
# example_session.behaviour_trials  # neural data during behaviour trials
assert example_session.behaviour_trials.ndim == 3
assert example_session.behaviour_trials.shape[0] == example_session.n_cells
assert example_session.behaviour_trials.shape[1] == example_session.n_trials
assert example_session.behaviour_trials.shape[2] == example_session.n_times

# example_session.pre_rew_trials  # neural data without behaviour (spontaneous reward deliveries at start of trial)
assert example_session.pre_rew_trials.ndim == 3
assert example_session.pre_rew_trials.shape[0] == example_session.n_cells
print(f'Session {example_session.signature} had {example_session.pre_rew_trials.shape[1]} reward only trials.')
assert example_session.pre_rew_trials.shape[2] == example_session.n_times



Session J064_R10 had 10 reward only trials.


In [6]:
## For Fig 1-3 of the manuscript, we normalize each trial/neuron with respect to their pre-stimulus baseline
## That is done by the following functions, which returns the normalized data, sorted, with some other options: 

(data_use_mat_norm, data_use_mat_norm_s1, data_use_mat_norm_s2, data_spont_mat_norm, ol_neurons_s1, ol_neurons_s2, outcome_arr,
        time_ticks, time_tick_labels, time_axis) = pop.normalise_raster_data(session=example_session, sorting_method='sum', sort_tt_list=['hit'],
                            sort_neurons=True, filter_150_stim=False)

For more information, see the notebooks in `popping-off/notebooks/Paper Figures/` that show what functions were used to create the figures of the Rowland ea manuscript.